In [49]:
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

In [50]:
PER_PACKET_COLS = ['_time', 'packet_interarrival_time', 'packet_jitter_raw', 'packet_jitter_weighted', 'packet_owd']
AGGREGATE_COLS  = ['_time', 'availability_2ms', 'availability_4ms', 'availability_8ms', 'availability_16ms', 'availability_32ms', 'availability_64ms', 'availability_128ms', 'packet_loss', 'throughput']

In [51]:
FILE_PATH = "/home/shared/validation_files/csvs/results"
df1, df2, df3 = pd.read_csv(f"{FILE_PATH}/analysis_validation_one.csv", skiprows=3),\
                pd.read_csv(f"{FILE_PATH}/analysis_validation_two.csv", skiprows=3),\
                pd.read_csv(f"{FILE_PATH}/analysis_validation_three.csv", skiprows=3)

df_aggregate1, df_aggregate2, df_aggregate3 = pd.read_csv(f"{FILE_PATH}/analysis_validation_one_aggregate.csv", skiprows=3),\
                pd.read_csv(f"{FILE_PATH}/analysis_validation_two_aggregate.csv", skiprows=3),\
                pd.read_csv(f"{FILE_PATH}/analysis_validation_three_aggregate.csv", skiprows=3)

In [52]:
df1 = df1[PER_PACKET_COLS]
df2 = df2[PER_PACKET_COLS]
df3 = df3[PER_PACKET_COLS]

df_aggregate1 = df_aggregate1[AGGREGATE_COLS]
df_aggregate2 = df_aggregate2[AGGREGATE_COLS]
df_aggregate3 = df_aggregate3[AGGREGATE_COLS]

In [66]:
df1.head()

,_time,packet_interarrival_time,packet_jitter_raw,packet_jitter_weighted,packet_owd
0,0.001,0.001,-0.012936,0.000808,0.001754
1,0.002,0.001,0.009247,0.001336,0.011001
2,0.003,0.001,-0.006694,0.001671,0.004307
3,0.004,0.001,0.011928,0.002312,0.016235
4,0.005,0.001,-0.001961,0.002290,0.014274


In [67]:
df_aggregate1.head()

,_time,availability_2ms,availability_4ms,availability_8ms,availability_16ms,availability_32ms,availability_64ms,availability_128ms,packet_loss,throughput
0,0.0,0.010,0.042,0.175,0.768,0.981,0.981,0.981,0.008,460288
1,1.0,0.018,0.047,0.204,0.765,0.985,0.985,0.985,0.007,460752
2,2.0,0.016,0.050,0.195,0.776,0.983,0.983,0.983,0.006,461216
3,3.0,0.014,0.050,0.183,0.792,0.982,0.983,0.983,0.010,459360
4,4.0,0.013,0.045,0.192,0.792,0.982,0.982,0.982,0.009,459824


In [64]:
def parse_timestamp(timestamp_str, with_fraction=True):
    if with_fraction:
        try:
            seconds_part = timestamp_str[:-1].split('.')[0]
            fraction_part = timestamp_str[:-1].split('.')[1]
        except:
            seconds_part = timestamp_str[:-1]
            fraction_part = 0
    else:
        seconds_part = timestamp_str[:-1]

    timestamp = datetime.strptime(seconds_part, '%Y-%m-%dT%H:%M:%S')

    return timestamp.timestamp() + (float(f"0.{fraction_part}") if with_fraction else 0)

In [65]:
df1['_time'] = df1['_time'].apply(parse_timestamp)
df2['_time'] = df2['_time'].apply(parse_timestamp)
df3['_time'] = df3['_time'].apply(parse_timestamp)

df_aggregate1['_time'] = df_aggregate1['_time'].apply(lambda x: parse_timestamp(x, False))
df_aggregate2['_time'] = df_aggregate2['_time'].apply(lambda x: parse_timestamp(x, False))
df_aggregate3['_time'] = df_aggregate3['_time'].apply(lambda x: parse_timestamp(x, False))

In [68]:
def analyze_iat(df):
    iat = df['packet_interarrival_time']
    mean, std = iat.mean(), iat.std()

    return mean, std

def analyze_owd(df):
    owd = df['packet_owd']
    mean, std = owd.mean(), owd.std()

    return mean, std

def analyze_packet_loss(df):
    packet_loss = df['packet_loss']
    return packet_loss.mean()

In [72]:
print("DF1 IAT")
print(f'Expected mean: 0.001, Calculated mean: {analyze_iat(df1)[0]}')
print(f'Expected std: 0.0, Calculated std: {analyze_iat(df1)[1]}')
print("\n")

print("DF2 IAT")
print(f'Expected mean: 0.001, Calculated mean: {analyze_iat(df2)[0]}')
print(f'Expected std: 0.0, Calculated std: {analyze_iat(df2)[1]}')
print("\n")

print("DF3 IAT")
print(f'Expected mean: 0.001, Calculated mean: {analyze_iat(df3)[0]}')
print(f'Expected std: 0.0, Calculated std: {analyze_iat(df3)[1]}')
print("\n")

DF1 IAT
Expected mean: 0.001, Calculated mean: 0.0009999999999999339
Expected std: 0.0, Calculated std: 7.551555220825162e-15


DF2 IAT
Expected mean: 0.001, Calculated mean: 0.0009999999999999339
Expected std: 0.0, Calculated std: 7.551555220825162e-15


DF3 IAT
Expected mean: 0.001, Calculated mean: 0.0009999999999999339
Expected std: 0.0, Calculated std: 7.551555220825162e-15




In [70]:
print("DF1 OWD")
print(f'Expected mean: 0.012, Calculated mean: {analyze_owd(df1)[0]}')
print(f'Expected std: 0.005, Calculated std: {analyze_owd(df1)[1]}')
print("\n")

print("DF2 OWD")
print(f'Expected mean: 0.032, Calculated mean: {analyze_owd(df2)[0]}')
print(f'Expected std: 0.025, Calculated std: {analyze_owd(df2)[1]}')
print("\n")

print("DF3 OWD")
print(f'Expected mean: 0.02, Calculated mean: {analyze_owd(df3)[0]}')
print(f'Expected std: 0.05, Calculated std: {analyze_owd(df3)[1]}')
print("\n")

DF1 OWD
Expected mean: 0.012, Calculated mean: 0.012117812457968141
Expected std: 0.005, Calculated std: 0.0048585732005709615


DF2 OWD
Expected mean: 0.032, Calculated mean: 0.03688567566652891
Expected std: 0.025, Calculated std: 0.02108108085956691


DF3 OWD
Expected mean: 0.02, Calculated mean: 0.0480105957784959
Expected std: 0.05, Calculated std: 0.03391145892307142




In [71]:

print("DF1 Packet loss")
print(f'Expected rate: 0.01, Calculated rate {analyze_packet_loss(df_aggregate1)}')
print("\n")

print("DF2 Packet loss")
print(f'Expected rate: 0.05, Calculated rate {analyze_packet_loss(df_aggregate2)}')
print("\n")

print("DF1 Packet loss")
print(f'Expected rate: 0.2, Calculated rate {analyze_packet_loss(df_aggregate3)}')
print("\n")

DF1 Packet loss
Expected rate: 0.01, Calculated rate 0.010155000022496051


DF2 Packet loss
Expected rate: 0.05, Calculated rate 0.04980999997816975


DF1 Packet loss
Expected rate: 0.2, Calculated rate 0.19955999962985516


